In [1]:
import src.data_utils as data_utils
import src.tree_build as dt_tree
import src.heurestics as dt_heur
import src.tree_utils as tree_utils
from pprint import pprint

import numpy as np
import pandas as pd

In [2]:
car_dataset_path = 'data/car.data'
car_dataset_labels = ['buying','maint','doors','persons','lug_boot','safety','CLASS']
car_dataset = data_utils.preapare_dataset(car_dataset_path, car_dataset_labels)
car_dataset.head()

,buying,maint,doors,persons,lug_boot,safety,CLASS
0,vhigh,med,4,2,small,med,unacc
1,vhigh,vhigh,2,more,small,low,unacc
2,low,med,5more,4,med,high,vgood
3,med,med,3,4,big,low,unacc
4,low,high,2,more,med,med,acc


In [3]:
cmc_dataset_path = 'data/cmc.data'
cmc_dataset_labels = ['age','education','husbandEducation','noChildren','religion','isWorking','husbandOccupation','livingStandard', 'mediaExposure', 'CLASS']
cmc_dataset = data_utils.preapare_dataset(cmc_dataset_path, cmc_dataset_labels)
cmc_dataset.head()

,age,education,husbandEducation,noChildren,religion,isWorking,husbandOccupation,livingStandard,mediaExposure,CLASS
0,44,3,4,5,1,0,1,4,0,3
1,29,3,4,2,1,1,3,4,0,1
2,44,2,4,4,1,1,3,3,0,2
3,35,2,3,6,1,0,3,2,0,3
4,41,3,2,11,1,1,2,3,0,1


In [4]:
cancer_dataset_path = 'data/breast-cancer.data'
cancer_dataset_labels = ['CLASS','age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast', 'breast-quad', 'irradiat']
cancer_dataset = data_utils.preapare_dataset(cancer_dataset_path, cancer_dataset_labels)
cancer_dataset = cancer_dataset[['age','menopause','tumor-size','inv-nodes', 'node-caps','deg-malig','breast', 'breast-quad', 'irradiat', 'CLASS']]
cancer_dataset.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,CLASS
0,50-59,ge40,20-24,0-2,no,2,right,left_up,no,no-recurrence-events
1,40-49,premeno,30-34,3-5,no,2,right,left_up,no,recurrence-events
2,40-49,premeno,25-29,0-2,no,3,left,right_up,no,recurrence-events
3,50-59,ge40,0-4,0-2,no,1,right,central,no,no-recurrence-events
4,50-59,lt40,30-34,0-2,no,3,right,left_up,no,no-recurrence-events


In [5]:
"""
Example of building of the decision tree
"""
car_training, car_testing = data_utils.data_split(car_dataset)

car_tree_gini= dt_tree.build_tree_generic(dt_heur.gini_impurity_weighted,car_training)
pprint(car_tree_gini)
tree_accuracy = tree_utils.tree_accuracy(car_testing, car_tree_gini)
print("Tree accuracy for testing data {0}%\n".format(tree_accuracy))

{<safety(0.38)>: high[457] low[464] med[461] : {'high': 'unacc',
                                                'low': 'unacc',
                                                'med': 'unacc'}}
Tree accuracy for testing data 71.09826589595376%



In [6]:
"""
TODO: Move to tree_testing_utils
"""
class DecisionTreeWithParameters():
    def __init__(self, decision_tree,used_heuristic,training_data_size,tree_accuracy_training,tree_accuracy_testing):
        self.decision_tree = decision_tree
        self.used_heuristic = used_heuristic.__name__
        self.training_data_size = training_data_size
        self.tree_accuracy_training = tree_accuracy_training
        self.tree_accuracy_testing = tree_accuracy_testing
        self.tree_levels = tree_utils.number_of_levels(decision_tree)
        self.tree_leafs = tree_utils.number_of_leafs(decision_tree)
    
    def __str__(self):
        return "Accuracy training={0:.2f}% testing={1:.2f}%, levels={2}, leafs={3}, training data={4}%".format(self.tree_accuracy_training, self.tree_accuracy_testing, self.tree_levels, self.tree_leafs, self.training_data_size)
        
        

In [7]:
"""
TODO: Move to tree_testing_utils
"""
def testing_data_with_given_size(data, training_data_size, heuristic):
    training_data, testing_data = data_utils.data_split(data, training_data_size)
    decision_tree = dt_tree.build_tree_generic(heuristic, training_data)
    tree_accuracy_training = tree_utils.tree_accuracy(training_data, decision_tree)
    tree_accuracy = tree_utils.tree_accuracy(testing_data, decision_tree)
    generated_decision_tree = DecisionTreeWithParameters(decision_tree, heuristic, training_data_size, tree_accuracy_training, tree_accuracy)
    print(generated_decision_tree)

In [8]:
"""
TODO: Move to tree_testing_utils
"""
def testing_data_with_given_heuristic(data, heuristic):
    sizes = [50, 60, 70, 80]
    print("Testing with heuristic={0}".format(heuristic.__name__))
    for training_data_size in sizes:
        testing_data_with_given_size(data, training_data_size, heuristic)

In [9]:
"""
TODO: Move to tree_testing_utils
Add heuristics as parameter
"""
def testing_given_dataset(data):
    heuristics = [dt_heur.gini_impurity_weighted, dt_heur.gini_pri, dt_heur.info_gain, 
                  dt_heur.information_gain_ratio, dt_heur.j_measure, dt_heur.relevance,
                  dt_heur.relief, dt_heur.weight_of_evidence, dt_heur.mdl_simple]
    
    for heuristic in heuristics:
        testing_data_with_given_heuristic(data, heuristic)
        
def testing_given_dataset_without_relief(data):
    heuristics = [dt_heur.gini_impurity_weighted, dt_heur.gini_pri, dt_heur.info_gain, 
                  dt_heur.information_gain_ratio, dt_heur.j_measure, dt_heur.relevance,
                  dt_heur.weight_of_evidence, dt_heur.mdl_simple]
    
    for heuristic in heuristics:
        testing_data_with_given_heuristic(data, heuristic)

In [10]:
testing_given_dataset(car_dataset)

Testing with heuristic=gini_impurity_weighted
Accuracy training=71.88% testing=69.91%, levels=4, leafs=5, training data=50%
Accuracy training=68.53% testing=72.25%, levels=2, leafs=3, training data=60%
Accuracy training=68.98% testing=72.45%, levels=2, leafs=3, training data=70%
Accuracy training=69.75% testing=71.10%, levels=2, leafs=3, training data=80%
Testing with heuristic=gini_pri
Accuracy training=67.82% testing=72.22%, levels=4, leafs=7, training data=50%
Accuracy training=68.53% testing=72.25%, levels=2, leafs=4, training data=60%
Accuracy training=68.98% testing=72.45%, levels=2, leafs=4, training data=70%
Accuracy training=69.75% testing=71.10%, levels=2, leafs=4, training data=80%
Testing with heuristic=info_gain
Accuracy training=67.82% testing=72.22%, levels=2, leafs=4, training data=50%
Accuracy training=68.53% testing=72.25%, levels=2, leafs=4, training data=60%
Accuracy training=68.98% testing=72.45%, levels=2, leafs=4, training data=70%
Accuracy training=69.75% testin

In [12]:
testing_given_dataset(cmc_dataset)

Testing with heuristic=gini_impurity_weighted
Tree accuracy training=43.48% testing=41.93%, levels=2, leafs=2, training data=50%
Tree accuracy training=43.15% testing=42.03%, levels=2, leafs=2, training data=60%
Tree accuracy training=43.06% testing=41.86%, levels=2, leafs=2, training data=70%
Tree accuracy training=42.61% testing=43.05%, levels=4, leafs=3, training data=80%
Testing with heuristic=gini_pri
Tree accuracy training=43.48% testing=41.93%, levels=2, leafs=2, training data=50%
Tree accuracy training=43.15% testing=42.03%, levels=2, leafs=2, training data=60%
Tree accuracy training=43.06% testing=41.86%, levels=2, leafs=2, training data=70%
Tree accuracy training=42.61% testing=43.05%, levels=2, leafs=2, training data=80%
Testing with heuristic=info_gain
Tree accuracy training=43.48% testing=41.93%, levels=2, leafs=2, training data=50%
Tree accuracy training=43.15% testing=42.03%, levels=2, leafs=2, training data=60%
Tree accuracy training=43.06% testing=41.86%, levels=2, lea

In [13]:
testing_given_dataset_without_relief(cancer_dataset)

Testing with heuristic=gini_impurity_weighted
Tree accuracy training=87.41% testing=50.35%, levels=10, leafs=44, training data=50%
Tree accuracy training=83.04% testing=47.83%, levels=10, leafs=51, training data=60%
Tree accuracy training=81.50% testing=56.98%, levels=10, leafs=46, training data=70%
Tree accuracy training=81.58% testing=55.17%, levels=10, leafs=57, training data=80%
Testing with heuristic=gini_pri
Tree accuracy training=69.93% testing=67.13%, levels=6, leafs=13, training data=50%
Tree accuracy training=70.76% testing=69.57%, levels=2, leafs=3, training data=60%
Tree accuracy training=70.00% testing=70.93%, levels=6, leafs=18, training data=70%
Tree accuracy training=70.61% testing=67.24%, levels=8, leafs=21, training data=80%
Testing with heuristic=info_gain
Tree accuracy training=69.93% testing=70.63%, levels=6, leafs=4, training data=50%
Tree accuracy training=70.76% testing=69.57%, levels=4, leafs=3, training data=60%
Tree accuracy training=70.00% testing=70.93%, le